In [3]:
# Python 3.x
import re
import math

def Input(day):
    "Open this day's input file."
    filename = 'inputs/{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        print("Oops, couldn't open input")


# Day 1 - Inverse Captcha
## Part 1

> The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

This could be done by reading the digits into an array, then reducing them. As a Python noob, I read that for clarity, list comprehensions are preferred over lambdas, e.g. see Guido van Rossum's [post](http://www.artima.com/weblogs/viewpost.jsp?thread=98196) from 2005. However, list comprehensions produce lists, whereas reduce produces a single output. Van Rossum concedes that `reduce()` may be used when the operator is associative, so I'm going to go ahead with `reduce()`.

In [4]:
from functools import reduce

In [5]:
def chars_to_int_array(str):
    "Given a string of digits, returns them as an array of integers"
    return [int(ch) for ch in str]

def circular_list_next_item(list, index, offset=1): 
    "Returns the list item that is 'offset' elements away after the one given by the index; the item after the last item is the first item"
    return list[(index + offset) % len(list)]

def inverse_captcha_1(arr):
    "Returns sum of digits in array that match the next digit, treating list as circular."
    return reduce(lambda sum, i: sum + (arr[i] if circular_list_next_item(arr, i) == arr[i] else 0), range(len(arr)), 0)
    

assert inverse_captcha_1(chars_to_int_array("1122")) == 3
assert inverse_captcha_1(chars_to_int_array("1111")) == 4
assert inverse_captcha_1(chars_to_int_array("1234")) == 0
assert inverse_captcha_1(chars_to_int_array("91212129")) == 9

inverse_captcha_1(chars_to_int_array(Input(1).read().rstrip("\n")))

1141

## Part 2

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. Fortunately, your list has an even number of elements

In [6]:
def inverse_captcha_2(arr):
    "Returns sum of digits in array that match the digit that is halfway around, treating list as circular."
    return reduce(lambda sum, i: sum + (arr[i] if circular_list_next_item(arr, i, len(arr)//2) == arr[i] else 0), range(len(arr)), 0)

assert inverse_captcha_2(chars_to_int_array("1212")) == 6
assert inverse_captcha_2(chars_to_int_array("1221")) == 0
assert inverse_captcha_2(chars_to_int_array("123425")) == 4
assert inverse_captcha_2(chars_to_int_array("123123")) == 12
assert inverse_captcha_2(chars_to_int_array("12131415")) == 4

inverse_captcha_2(chars_to_int_array(Input(1).read().rstrip("\n")))

950

# Day 2

## Part 1
> The spreadsheet consists of rows of apparently-random numbers. To make sure the recovery process is on the right track, they need you to calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

Initially, I thought I could iterate over each row, keep track of the max and min and subtract. But it's a lot easier to just call the built-in `max` and `min`. Both methods have `O(n)` running time.

In [39]:
def line_to_int_array(line):
    "Given a string with whitespace-separated numbers, parse it as an array of integers"
    return [int(n) for n in line.split()]

def lines_to_int_arrays(lines):
    "Given a multi-line string, each with whitespace-separated numbers, return an array of arrays of integers"
    return [line_to_int_array(L) for L in lines]
    
def max_subtract_min(L):
    "Return the largest element minus the smallest element"
    return max(L) - min(L)

assert(sum([max_subtract_min(L) for L in lines_to_int_arrays("5 1 9 5\n7 5 3\n2 4 6 8".split("\n"))])) == 18

sum([max_subtract_min(L) for L in lines_to_int_arrays(Input(2).readlines())])

37923

## Part 2
> It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

I had a lot of ideas of how to solve this. Clearly, we need to consider all pairs of numbers within each row. Sorting each row's numbers first, would simplify things a little, because we could always divide the smaller into the larger, by having two loops, one from each end. Or, perhaps we could apply a modulo operator, look for the single entry with zero, but that's wasteful, as we don't abort early. 

In the end, I preferred the simpler double for-loop, and having to compute the max/min within the inner block, since it didn't requiring any sorting, or reversing.

In [45]:
def quotient_of_divisible_elements(L):
    "Given a list of numbers in which there are two numbers where one exactly divides the other, return the quotient between them."
    for i in range(0, len(L)-1):
        for j in range(i+1, len(L)):
            p = max(L[i], L[j])
            q = min(L[i], L[j])
            if p % q == 0:
                return p // q

assert(sum([quotient_of_divisible_elements(L) for L in lines_to_int_arrays("5 9 2 8\n9 4 7 3\n3 8 6 5".split("\n"))])) == 9

sum([quotient_of_divisible_elements(L) for L in lines_to_int_arrays(Input(2).readlines())])

263

# Day 3
## Part 1
Note that the bottom right corner is always the square of an odd number.

In [5]:
print((559*559)-312051)
print(559//2)


430
279


312051 sits somewhere on the bottom row of a grid that is 559 x 559
It has to move floor(559/2)=279 steps to get to the row where 1 is
If the number was 312481 (559*559) then, it would have to move 279 steps left to get to column where 1 is.
If the number was 312481 - floor(559/2) = 312202, it would already be at the column
If the number was (312202 - 1) = 312201, it would have to move one to the right
The number is (312202 - 151) = 312051 it would have to move 151 to the right
So answer is 151 + 279 = 430

## Part 2
> If we have a function that can tell us the (x,y) of each item, given the square's index, we could easily compute the values by filling them in sequentially, as we could easily get the neighbours from a 2D array.

Do the values fit a recurrence relation?
Could we figure out a way to map a straight 1d array and coil it?

The size of each concentric "ring" is:
1, 8=2*4, 16=4*4, 24=6*4


1 R 
2 U 3 L 4 L 5 D 6 D 7 R 8 R 9 R
10 U 11 U 12 U 13 L 14 L 15 L 16 L 17 D 18 D 19 D 20 D 21 R 22 R 23 R 24 R 25 R

Make a really large array, e.g. 559x559. Start in middle. Then keep track of size of spiral we're currently filling, and then fill it.



In [56]:
def initialize_2d(rows, cols, init_value = 0):
    return [ [init_value] * cols for r in range(rows) ]

def neighbour_sum(arr, x, y):
    "Return the sum of the adjacent neighbours of arr[x][y], including diagonals"
    rows = len(arr)
    cols = len(arr[0])
    
    if x == (rows // 2) and y == (cols // 2):
        return 1
    
    sum = 0
    if (x-1 >= 0):
        sum += arr[x-1][y]
        if (y-1 >= 0):
            sum += arr[x-1][y-1]
        if (y+1 <= rows-1):
            sum += arr[x-1][y+1]
    if (x+1 <= cols-1):    
        sum += arr[x+1][y]
        if (y-1 >- 0):
            sum += arr[x+1][y-1]
        if (y+1 <= rows-1):
            sum += arr[x+1][y+1]

    if (y-1 >- 0):
        sum += arr[x][y-1]
    if (y+1 <= rows-1):
        sum += arr[x][y+1]

    return sum

In [68]:
def spiral_index_generator(dim):
    generated = 0
    
    x = dim // 2
    y = dim // 2
    yield x, y

    for spiral_len in range(3, dim+1, 2):
        # Each time we start new ring, we are one up
        # from bottom right corner
        dir_steps = 1
        x += 1
        for curdir in ['U', 'L', 'D', 'R']:
            while dir_steps < spiral_len-1:
                yield x, y   

                if curdir == 'U':
                    y -= 1
                elif curdir == 'L':
                    x -= 1
                elif curdir == 'D':
                    y += 1
                elif curdir == 'R':
                    x += 1
                dir_steps += 1
                
            dir_steps = 0
        yield x, y

def stress_test(dim, sentinel):
    arr = initialize_2d(dim, dim)
    for x, y, in spiral_index_generator(dim):
        arr[x][y] = neighbour_sum(arr, x, y)
        if arr[x][y] > sentinel:
            return arr[x][y]

stress_test(15, 312051)

312453